In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
X_train=pd.read_csv("train.csv")
X_test=pd.read_csv("test.csv")

In [3]:
X_train.head()

,Elevation(meters),Aspect(degrees),Slope(degrees),Horizontal_Distance_To_Hydrology(meters),Vertical_Distance_To_Hydrology(meters),Horizontal_Distance_To_Roadways(meters),Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points(meters),Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2982,186,14,323,66,5351,222,249,158,1530,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5
1,2929,243,15,335,33,5778,189,251,198,6429,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,7
2,3051,162,12,1224,253,3182,231,243,142,1167,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5
3,3090,345,17,362,92,2460,184,211,164,2384,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
4,3023,4,9,234,51,4994,208,222,151,3813,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,7


In [4]:
X_test.head()

,Elevation(meters),Aspect(degrees),Slope(degrees),Horizontal_Distance_To_Hydrology(meters),Vertical_Distance_To_Hydrology(meters),Horizontal_Distance_To_Roadways(meters),Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points(meters),Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40
0,2798,151,12,60,-5,3373,236,240,132,6083,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3582,96,34,420,80,2579,250,170,19,3771,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2619,171,12,270,-52,2764,228,245,147,2008,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2897,155,5,210,14,2437,226,240,149,2455,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2845,32,12,309,47,2778,217,213,129,42,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y_train=X_train.iloc[: ,-1]

In [6]:
X_train=X_train.iloc[: , :-1]

In [7]:
X_train.head()

,Elevation(meters),Aspect(degrees),Slope(degrees),Horizontal_Distance_To_Hydrology(meters),Vertical_Distance_To_Hydrology(meters),Horizontal_Distance_To_Roadways(meters),Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points(meters),Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_19,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40
0,2982,186,14,323,66,5351,222,249,158,1530,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2929,243,15,335,33,5778,189,251,198,6429,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,3051,162,12,1224,253,3182,231,243,142,1167,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3090,345,17,362,92,2460,184,211,164,2384,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3023,4,9,234,51,4994,208,222,151,3813,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [8]:
y_train.head()

0    5
1    7
2    5
3    7
4    7
Name: Cover_Type, dtype: int64

In [9]:
X_train.shape

(29050, 54)

In [10]:
null_columns=X_train.columns[X_train.isnull().any()]
X_train[null_columns].isnull().sum()


Series([], dtype: float64)

In [11]:
null_columns1=X_test.columns[X_test.isnull().any()]
X_test[null_columns1].isnull().sum()

Wilderness_Area_1    1
Wilderness_Area_2    1
Wilderness_Area_3    1
Wilderness_Area_4    1
Soil_Type_1          1
Soil_Type_2          1
Soil_Type_3          1
Soil_Type_4          1
Soil_Type_5          1
Soil_Type_6          1
Soil_Type_7          1
Soil_Type_8          1
Soil_Type_9          1
Soil_Type_10         1
Soil_Type_11         1
Soil_Type_12         1
Soil_Type_13         1
Soil_Type_14         1
Soil_Type_15         1
Soil_Type_16         1
Soil_Type_17         1
Soil_Type_18         1
Soil_Type_19         1
Soil_Type_20         1
Soil_Type_21         1
Soil_Type_22         1
Soil_Type_23         1
Soil_Type_24         1
Soil_Type_25         1
Soil_Type_26         1
Soil_Type_27         1
Soil_Type_28         1
Soil_Type_29         1
Soil_Type_30         1
Soil_Type_31         1
Soil_Type_32         1
Soil_Type_33         1
Soil_Type_34         1
Soil_Type_35         1
Soil_Type_36         1
Soil_Type_37         1
Soil_Type_38         1
Soil_Type_39         1
Soil_Type_4

In [12]:
X_test[null_columns1] = X_test[null_columns1].replace(np.nan, 0)

# inplace
X_test[null_columns1].replace(np.nan, 0, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 4)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [15]:
from xgboost import XGBClassifier
xgb=XGBClassifier(n_estimators=1000,learning_rate=0.5,max_depth=2,objective='multiclass',metric='multi_logloss')
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=2,
              metric='multi_logloss', min_child_weight=1, missing=None,
              n_estimators=1000, n_jobs=1, nthread=None,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [16]:
y_pred=xgb.predict(X_train)
y_pred

array([5, 5, 5, ..., 5, 5, 7])

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.826987951807229

In [18]:
y_pred1=xgb.predict(X_test)
y_pred1

array([5, 1, 5, ..., 5, 5, 5])

In [19]:
y_prob=xgb.predict_proba(X_test)

In [20]:
submission=pd.DataFrame(y_prob)
submission.head()

,0,1,2,3,4,5,6
0,3.858487e-07,3.376619e-04,1.507921e-08,7.428790e-06,9.601325e-01,2.612351e-04,0.039261
1,9.988652e-01,3.609532e-08,8.335095e-12,5.108531e-12,6.605524e-08,9.656070e-12,0.001135
2,1.051889e-05,2.224386e-02,1.210739e-02,4.092742e-07,9.083558e-01,3.525981e-02,0.022022
3,4.836766e-08,7.389737e-03,4.063508e-07,6.725112e-06,9.653112e-01,3.078072e-05,0.027261
4,1.540578e-04,2.396741e-03,4.917940e-04,1.755216e-09,2.462295e-01,6.706156e-05,0.750661


In [21]:
submission[submission<0.05]=0
submission[submission>0.95]=1
submission.head()

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000
1,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.908356,0.0,0.000000
3,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.246230,0.0,0.750661


In [22]:
submission.to_excel('forest.xlsx', index=False)

In [23]:
submission1=submission.round(1)

In [24]:
submission.head()

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000
1,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.908356,0.0,0.000000
3,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.246230,0.0,0.750661


In [25]:
submission1.to_excel('forest1.xlsx', index=False)